## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
#from google.colab import drive
#drive.mount('/gdrive')

In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [6]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [7]:
data.drop(['date', 'symbol'], axis = 1, inplace = True)

In [8]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [9]:
data.shape

(851264, 5)

In [10]:
data = data.iloc[:1000]

In [11]:
data.shape

(1000, 5)

In [12]:
data['volume'] = data['volume'] /1000000

In [13]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


In [14]:
X = data.drop('volume', axis = 1)
y = data['volume']

### Divide the data into train and test sets

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.30, random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [17]:
train_x = train_x.astype('float32')
train_y = train_y.astype('float32')
test_x = test_x.astype('float32')
test_y = test_y.astype('float32')

In [18]:
print(train_x.shape)
print(train_y.shape)

(700, 4)
(700,)


### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [19]:
from sklearn.preprocessing import Normalizer

In [20]:
transformer = Normalizer()
train_x = transformer.fit_transform(train_x)

In [21]:
train_x[0]

array([0.50184363, 0.49967504, 0.4953379 , 0.5031086 ], dtype=float32)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [22]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [23]:
def prediction(x, w, b):
    xw_matmul= tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    return y

3.Loss (Cost) Function [Mean square error]

In [24]:
def loss(y_actual, y_predicted):
    diff = y_actual-y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg
    

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [27]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [28]:
train_y = np.array(train_y)

In [29]:
for i in range(100):
    
    w, b = train(train_x, train_y, w, b)
    print('Current Loss on iteration', i, loss(train_y, prediction(train_x, w, b)).numpy())

Current Loss on iteration 0 223.49039
Current Loss on iteration 1 221.28828
Current Loss on iteration 2 219.259
Current Loss on iteration 3 217.38876
Current Loss on iteration 4 215.66518
Current Loss on iteration 5 214.07668
Current Loss on iteration 6 212.61267
Current Loss on iteration 7 211.26349
Current Loss on iteration 8 210.02014
Current Loss on iteration 9 208.8744
Current Loss on iteration 10 207.81828
Current Loss on iteration 11 206.84483
Current Loss on iteration 12 205.94783
Current Loss on iteration 13 205.12126
Current Loss on iteration 14 204.35939
Current Loss on iteration 15 203.65736
Current Loss on iteration 16 203.01028
Current Loss on iteration 17 202.41388
Current Loss on iteration 18 201.86418
Current Loss on iteration 19 201.35777
Current Loss on iteration 20 200.89095
Current Loss on iteration 21 200.46075
Current Loss on iteration 22 200.06422
Current Loss on iteration 23 199.69884
Current Loss on iteration 24 199.36218
Current Loss on iteration 25 199.05173

### Get the shapes and values of W and b

In [30]:
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[1.3549175]
 [1.3598167]
 [1.3410656]
 [1.3713948]]
Bias:
 [2.7137554]


In [31]:
w.shape

TensorShape([4, 1])

In [32]:
b.shape

TensorShape([1])

### Model Prediction on 1st Examples in Test Dataset

In [33]:
test_x = transformer.fit_transform(test_x)

In [34]:
prediction(test_x[0:1], w, b).numpy()

array([[5.427437]], dtype=float32)

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [35]:
iris = pd.read_csv('iris-2.csv')

In [36]:
iris.shape

(150, 6)

In [37]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [38]:
iris = iris.drop('Id', axis = 1)

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [39]:
iris = pd.get_dummies(iris)

In [40]:
iris.shape

(150, 7)

In [41]:
iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [60]:
X_iris = iris[["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"]]
Y_iris = iris[["Species_Iris-setosa","Species_Iris-versicolor","Species_Iris-virginica"]]

In [61]:
X_iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [62]:
Y_iris.head()

,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [42]:
x = iris.iloc[:, 0:4]

In [43]:
x.shape

(150, 4)

In [44]:
x.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [45]:
y = iris.iloc[:, 4:8]

In [46]:
y.shape

(150, 3)

In [47]:
y.head()

,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [48]:
tf.keras.backend.clear_session()

In [64]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

#Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, input_shape=(4,), activation='softmax'))

#Compile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Model Training 

In [63]:
trainX, testX, trainY, testY = train_test_split(X_iris, Y_iris, test_size=0.30, random_state=1)

In [70]:
model.fit(trainX.values, trainY.values, validation_data=(testX.values, testY.values), epochs=100, batch_size=32)

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 1ms/sample - loss: 0.3424 - accuracy: 0.9143 - val_loss: 0.4376 - val_accuracy: 0.8000
Epoch 2/100
105/105 [==============================] - 0s 248us/sample - loss: 0.3539 - accuracy: 0.9143 - val_loss: 0.4355 - val_accuracy: 0.8000
Epoch 3/100
105/105 [==============================] - 0s 253us/sample - loss: 0.3335 - accuracy: 0.8952 - val_loss: 0.4336 - val_accuracy: 0.8000
Epoch 4/100
105/105 [==============================] - 0s 248us/sample - loss: 0.3483 - accuracy: 0.9143 - val_loss: 0.4317 - val_accuracy: 0.8000
Epoch 5/100
105/105 [==============================] - 0s 257us/sample - loss: 0.3358 - accuracy: 0.9238 - val_loss: 0.4300 - val_accuracy: 0.8000
Epoch 6/100
105/105 [==============================] - 0s 267us/sample - loss: 0.3437 - accuracy: 0.9048 - val_loss: 0.4280 - val_accuracy: 0.8000
Epoch 7/100
105/105 [==============================] - 0s 281us/sample - lo

105/105 [==============================] - 0s 257us/sample - loss: 0.3097 - accuracy: 0.8952 - val_loss: 0.3560 - val_accuracy: 0.8222
Epoch 57/100
105/105 [==============================] - 0s 276us/sample - loss: 0.2825 - accuracy: 0.9619 - val_loss: 0.3557 - val_accuracy: 0.8222
Epoch 58/100
105/105 [==============================] - 0s 267us/sample - loss: 0.2687 - accuracy: 0.9714 - val_loss: 0.3555 - val_accuracy: 0.8222
Epoch 59/100
105/105 [==============================] - 0s 248us/sample - loss: 0.2866 - accuracy: 0.8952 - val_loss: 0.3550 - val_accuracy: 0.8222
Epoch 60/100
105/105 [==============================] - 0s 210us/sample - loss: 0.3408 - accuracy: 0.9048 - val_loss: 0.3535 - val_accuracy: 0.8222
Epoch 61/100
105/105 [==============================] - 0s 214us/sample - loss: 0.2881 - accuracy: 0.9238 - val_loss: 0.3531 - val_accuracy: 0.8222
Epoch 62/100
105/105 [==============================] - 0s 210us/sample - loss: 0.2815 - accuracy: 0.9143 - val_loss: 0.3519 

### Model Prediction

In [73]:
model.predict(testX.values)

array([[9.78208899e-01, 2.15605777e-02, 2.30598525e-04],
       [4.74254221e-01, 4.98401999e-01, 2.73438469e-02],
       [2.54570190e-02, 6.99617565e-01, 2.74925351e-01],
       [9.84108090e-01, 1.55619290e-02, 3.30060342e-04],
       [4.66341226e-05, 3.72525863e-02, 9.62700784e-01],
       [1.76706165e-02, 3.06248963e-01, 6.76080465e-01],
       [3.95113323e-03, 1.98172674e-01, 7.97876120e-01],
       [9.60872710e-01, 3.69857773e-02, 2.14155251e-03],
       [9.59046245e-01, 4.04806808e-02, 4.73105290e-04],
       [5.93580771e-04, 7.61070624e-02, 9.23299372e-01],
       [6.56559765e-02, 6.70441329e-01, 2.63902724e-01],
       [9.67127979e-01, 3.16572450e-02, 1.21482892e-03],
       [3.57631594e-04, 1.37699127e-01, 8.61943185e-01],
       [2.77063176e-02, 4.82170373e-01, 4.90123212e-01],
       [3.08733284e-02, 5.29600501e-01, 4.39526230e-01],
       [9.82957661e-01, 1.68668143e-02, 1.75494191e-04],
       [6.93752542e-02, 8.00573707e-01, 1.30051091e-01],
       [4.03329879e-02, 3.70212

### Save the Model

In [74]:
model.save('iris.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?